In [5]:
import weaviate
import os
from weaviate import classes as wvc

client = weaviate.connect_to_local(
    headers={
        "X-Openai-Api-Key": os.environ.get("OPENAI_APIKEY")
    }
)
print("Connected to Weaviate", client.is_ready())

Connected to Weaviate True


In [76]:
# Vamos apagar a coleção primeiro, caso exista
client.collections.delete("Produtos")
# e criar uma nova coleção
client.collections.create(
    name="Produtos",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-large"
    ),
    # por padrão é gpt-3.5-turbo, mas pode mudar pra gpt-4, por exemplo
    generative_config=wvc.config.Configure.Generative.openai(model="gpt-3.5-turbo")
)

In [ ]:
# agora vamos criar uma função pra transformar aquivos
import base64

def toBase64(path):
    with open(path, 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')

In [77]:
# vamos importar nossos produtos
produtos = [
    {"id":1, "title": "Cuscuz com Camarão", "description": "Delicioso cuscuz feito com camarão, ervilhas e palmito"},
    {"id":2, "title": "Cuscuz com Carne de Sol", "description": "A tradicional carne de sol, com o tradicional cuscuz feito carinho e muito tempero."},
    {"id":3, "title": "Cuscuz Fit", "description": "Excelente pro pós treino. Milho, cenoura, cebola, brócolis e muita saúde! sem carne."},
    {"id":4, "title": "Bolo de Rolo", "description": "O famoso bolo de rolo de goiabada cascão, doce como a vida, com mais de 10 camadas! Coberto com açúcar orgânico. Acompanha suco de manga rosa."},
    {"id":5, "title": "Buchada", "description": "Um dos pratos mais famosos da culinária pernambucana, especialmente na região do sertão. É feito com bucho de bode ou carneiro, tripas, picadinho de sangue coagulado e fígado, além de cebola, alho, limão, hortelã e temperos variados"},
    {"id":6, "title": "Cuscuz com Lagosta", "description": "Duas coisas gostosas juntas. Lagosta e Cuscuz! É de comer ajoelhado!"},
    {"id":7, "title": "Cartola", "description": "A cartola é um doce/sobremesa brasileiro, típico de Pernambuco. É uma das mais tradicionais sobremesas pernambucanas.bananas fritas em manteiga ou óleo, queijo de coalho ou de manteiga derretidos colocados em cima das bananas fritas, e cobertura, que pode ser a mais tradicional de canela e açúcar ou calda de chocolate"},
    
]

In [78]:
# vamos usar UUID deterministicos, "amarrando" o id do produto com um id dentro do Weaviate

from weaviate.util import generate_uuid5

with client.batch.dynamic() as batch:
    for produto in produtos:
        produto_id = produto.get("id")
        uuid = generate_uuid5(produto_id)
        batch.add_object(
            uuid=uuid,
            collection="Produtos",
            properties={
                "title": produto.get("title"),
                "description": produto.get("description"),
                #"image": toBase64(f"./produtos/produto{produto_id}.jpg")
            }
        )
# check if we had errors
print(client.batch.failed_objects)
print(client.batch.results.objs)

[]
BatchObjectReturn(_all_responses=[UUID('b04965e6-a9bb-591f-8f8a-1adcb2c8dc39'), UUID('4b166dbe-d99d-5091-abdd-95b83330ed3a'), UUID('98123fde-012f-5ff3-8b50-881449dac91a'), UUID('6ed955c6-506a-5343-9be4-2c0afae02eef'), UUID('c8691da2-158a-5ed6-8537-0e6f140801f2'), UUID('a6c4fc8f-6950-51de-a9ae-2c519c465071'), UUID('a9f96b98-dd44-5216-ab0d-dbfc6b262edf')], elapsed_seconds=0.0, errors={}, uuids={0: UUID('b04965e6-a9bb-591f-8f8a-1adcb2c8dc39'), 1: UUID('4b166dbe-d99d-5091-abdd-95b83330ed3a'), 2: UUID('98123fde-012f-5ff3-8b50-881449dac91a'), 3: UUID('6ed955c6-506a-5343-9be4-2c0afae02eef'), 4: UUID('c8691da2-158a-5ed6-8537-0e6f140801f2'), 5: UUID('a6c4fc8f-6950-51de-a9ae-2c519c465071'), 6: UUID('a9f96b98-dd44-5216-ab0d-dbfc6b262edf')}, has_errors=False)


In [79]:
collection = client.collections.get("Produtos")
client.collections.get("Produtos").aggregate.over_all()

AggregateReturn(properties={}, total_count=7)

In [ ]:
query = collection.query.near_text(
    "eu quero comer algo com frutos do mar", 
    return_metadata=wvc.query.MetadataQuery(distance=True),
    auto_limit=1
)
for obj in query.objects:
    print("#"*10)
    print(obj.properties.get("title"))
    print(obj.metadata.distance)

##########
Cuscuz com Lagosta
0.49463915824890137
##########
Cuscuz com Camarão
0.5260534286499023


In [ ]:
# agora, vamos gerar uma resposta com os dois pratos, e ainda sugerir uma bebida.
# veja que agora ao invés de collection.query, usaremos collection.generate

# tente agora algumas perguntas
query = "Eu quero comer algo com frutos do mar!"
#query = "Eu quero uma sobremesa"

query = collection.generate.near_text(
    # dica ninja: use query em modo minúsculo. De algum modo isso ajuda (não sei como)
    query=query.lower(), 
    return_metadata=wvc.query.MetadataQuery(distance=True),
    # ao adicionar auto_limit=1, nós escolhemos somente o primeiro
    # grupo semântico. A partir do momento que o próximo resultado tem um salto
    # grande na disância ou no score, ele passa a ser grupo 2, 3 e assim adiante
    auto_limit=1,
    grouped_task='''Você atende como garçonete na Loja de Cuscuz. Responda somente usando o contexto fornecido. 
    sugira os pratos encontrados. Comente cada um dos pratos, falando seu nome nome e descrição, 
    e para cada um sugira também uma bebida que harmonize com o prato ou atenda o pedido.
    Use expressões nordestinas, e seja nordestina: alegre, simpática e criativa.
    As opções de bebidas do dia a serem sugeridas:
    - Suco de Umbu
    - Suco de Cajá
    - suco de Manga Rosa
    - Capirinha de Carne de Cajú
    - Capirinha de Siriguela
    - Vinho Branco (alcólico)
    - Vinho Tinto (alcólico)
    - Cerveja IPA (alcólico)
    - Cerveja Pilsen (alcólico)
    - Cachaça (alcólico)
    '''
)
print(query.generated)

Oi meu bem, na Loja de Cuscuz temos dois pratos maravilhosos para te sugerir! Primeiramente, temos o "Cuscuz com Lagosta", que é uma combinação divina de lagosta com cuscuz, de dar água na boca! Para harmonizar, sugiro um suco de Cajá, que vai combinar perfeitamente com a leveza e sabor desse prato.

E também temos o "Cuscuz com Camarão", um delicioso cuscuz feito com camarão, ervilhas e palmito, uma explosão de sabores! Para acompanhar, que tal uma Capirinha de Siriguela? A acidez da siriguela vai realçar ainda mais o sabor do camarão. 

Espero que goste das sugestões, meu bem! Estamos aqui para te servir com alegria e sabor nordestino!


In [84]:
query = collection.query.near_text(
    "eu quero uma sobremesa", return_metadata=wvc.query.MetadataQuery(distance=True),
    auto_limit=1
)
for obj in query.objects:
    print("#"*10)
    print(obj.properties.get("title"))
    print(obj.metadata)

##########
Bolo de Rolo
MetadataReturn(creation_time=None, last_update_time=None, distance=0.5512862801551819, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None)
##########
Cartola
MetadataReturn(creation_time=None, last_update_time=None, distance=0.5748428106307983, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None)


In [104]:
query = collection.query.near_text(
    "Estou de Dieta, e quero algo com poucas calorias", return_metadata=wvc.query.MetadataQuery(distance=True),
    auto_limit=1
)
for obj in query.objects:
    print("#"*10)
    print(obj.properties.get("title"))
    print(obj.metadata)

##########
Cuscuz Fit
MetadataReturn(creation_time=None, last_update_time=None, distance=0.5523396730422974, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None)
